In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.metrics import f1_score, confusion_matrix

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

Using TensorFlow backend.


In [4]:
data = pd.read_csv('reviews_cleaned')


In [5]:
data.head()

,review,sentiment,cleaned
0,One of the other reviewers has mentioned that ...,1,reviewer mention watch oz episode hook right e...
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,1,think wonderful way spend time hot summer week...
3,Basically there's a family where a little boy ...,0,basically family little boy think zombie close...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,love visually stunning film watch offer vivid ...


In [6]:
data.isna().sum()

review       0
sentiment    0
cleaned      0
dtype: int64

In [18]:
#data.dropna(inplace =True)


In [20]:
data.head()

,review,sentiment,cleaned
0,One of the other reviewers has mentioned that ...,1,reviewer mention watch oz episode hook right e...
1,A wonderful little production. <br /><br />The...,1,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,1,think wonderful way spend time hot summer week...
3,Basically there's a family where a little boy ...,0,basically family little boy think zombie close...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,love visually stunning film watch offer vivid ...


In [7]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned'], data['sentiment'], 
                                                    random_state = 42, test_size =0.2, stratify= data['sentiment'])

In [ ]:
max_features = 5000
maxlen = 400

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

x_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
x_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

In [ ]:
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 1


model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))

model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))


# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))



C:\Users\jcmor\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40000 samples, validate on 10000 samples
Epoch 1/1
20960/40000 [==============>...............] - ETA: 2:10 - loss: 0.4337 - accuracy: 0.7867

In [ ]:
prediction = model.predict(x_test)

In [ ]:

y_pred = (prediction > 0.5)
from sklearn.metrics import f1_score, confusion_matrix
print(f'F1-score: {f1_score(y_pred, y_test)}')
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)